In [1]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("/mnt/cluster_storage/vicuna-13b/")

In [2]:
NUM_WORKERS = 16
BATCH_SIZE_PER_WORKER = 4

In [3]:
import ray
import os

@ray.remote(num_gpus=1)
def download_vicuna_13b(rank):
    if not os.path.exists("/tmp/vicuna-13b"):
        print(f"{rank}: Downloading vicuna model")
        os.system("aws s3 sync s3://large-dl-models-mirror/restricted/models--lmsys--vicuna-13b-delta-v1.1/main-safetensors/ /tmp/vicuna-13b >NUL 2>&1")
    print(f"{rank}: Finished")
    return True


tasks = [download_vicuna_13b.remote(i) for i in range(NUM_WORKERS)]
ray.get(tasks)

2023-06-23 23:38:59,574	INFO worker.py:1426 -- Connecting to existing Ray cluster at address: 10.0.104.253:6379...
2023-06-23 23:38:59,639	INFO worker.py:1607 -- Connected to Ray cluster. View the dashboard at https://session-vzyh3916u4zwmf1es6fazmbrgm.i.anyscaleuserdata-staging.com 
2023-06-23 23:38:59,642	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_71de2ab32420c3943f2aef46eb349233.zip' (0.05MiB) to Ray cluster...
2023-06-23 23:38:59,643	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_71de2ab32420c3943f2aef46eb349233.zip'.


(download_vicuna_13b pid=5790, ip=10.0.123.219) 3: Finished


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
config = AutoConfig.from_pretrained("/tmp/vicuna-13b")
HIDDEN_SIZE = config.hidden_size

In [5]:
import ray
from ray.air.config import CheckpointConfig, RunConfig, ScalingConfig
import torch
import transformers

from datasets import load_dataset
from ray.data.preprocessors import BatchMapper, Chain
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from deepspeed.ops.adam import DeepSpeedCPUAdam

# MODEL_NAME = "lmsys/vicuna-13b-delta-v1.1"
# MODEL_NAME = "EleutherAI/gpt-j-6B"
# MODEL_NAME = "tiiuae/falcon-7b"
# MODEL_NAME = "mosaicml/mpt-7b"
MODEL_NAME = "/tmp/vicuna-13b"

# hf_dataset = load_dataset("xtreme", "MLQA.en.en")
hf_dataset = load_dataset("cosmos_qa")["train"]
ray_dataset = ray.data.from_huggingface(hf_dataset).limit(8000)

PROMPT_TEMPLATE = """
Context: {context}
Question: {question}
Based on the context, the answer to the question would be: {answer} </s>
"""

def fill_prompt(batch):
    batch["input_sentence"] = batch.apply(
        lambda row: PROMPT_TEMPLATE.format(
            context=row["context"],
            question=row["question"],
            answer=row["answer0"]
        ),
        axis=1,
    )
    return batch[["input_sentence"]]


def tokenize(batch):
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    ret = tokenizer(
        list(batch["input_sentence"]),
        truncation=True,
        max_length=180,
        padding="max_length",
        return_tensors="np",
    )
    ret["labels"] = ret["input_ids"].copy()
    return dict(ret)


prompt_mapper = BatchMapper(fill_prompt, batch_format="pandas")
tokenize_mapper = BatchMapper(tokenize, batch_format="pandas")
preprocessor = Chain(prompt_mapper, tokenize_mapper)

import torch
import pytorch_lightning as pl
import transformers.deepspeed


class ZeRO3Config:
    def __init__(self, pl_module):
        self.config = pl_module.trainer.strategy.config

    def __call__(self, *args, **kwargs):
        return self

    def is_zero3(self) -> bool:
        return True


def enable_transformers_pretrained_deepspeed_sharding(
    pl_module: "pl.LightningModule",
) -> None:
    transformers.deepspeed._hf_deepspeed_config_weak_ref = ZeRO3Config(pl_module)

from accelerate import init_empty_weights


class Falcon7BModel(pl.LightningModule):
    def __init__(self, inference=False):
        super().__init__()
        torch.backends.cuda.matmul.allow_tf32 = True
        if inference:
            with init_empty_weights():
                config = AutoConfig.from_pretrained(MODEL_NAME, trust_remote_code=True)
                self.model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
            self.model.tie_weights()
        # else:
        #     enable_transformers_pretrained_deepspeed_sharding(self)
        #     self.model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True)
        #     self.model.tie_weights()
        
    def setup(self, stage) -> None:
        if not hasattr(self, "model"):
            print("Config :", self.trainer.strategy.config)
            enable_transformers_pretrained_deepspeed_sharding(self)
            self.model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True)
            print(self.model)

    def forward(self, batch):
        outputs = self.model(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
        )
        return outputs.loss

    def training_step(self, batch, batch_idx):
        torch.cuda.empty_cache()
        loss = self.forward(batch)
        self.log("train_loss", loss, prog_bar=True, on_step=True, sync_dist=True)
        return loss

    def configure_optimizers(self):
        return DeepSpeedCPUAdam(self.parameters(), lr=1e-5, fp32_optimizer_states=False)

[2023-06-23 23:39:03,430] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Found cached dataset cosmos_qa (/home/ray/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157)


  0%|          | 0/3 [00:00<?, ?it/s]

2023-06-23 23:39:06,191	WARNING dataset.py:249 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.

Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode


In [6]:
from pytorch_lightning.callbacks import TQDMProgressBar


# Create a customized progress bar for LightningTrainer
class FalconProgressBar(TQDMProgressBar):
    def __init__(self, num_iters_per_epoch, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_iters_per_epoch = num_iters_per_epoch

    def on_train_epoch_start(self, trainer, *_):
        super().on_train_epoch_start(trainer, *_)
        self.train_progress_bar.reset(self.num_iters_per_epoch)


total_batches = ray_dataset.count()
num_iters_per_epoch = total_batches // (NUM_WORKERS * BATCH_SIZE_PER_WORKER)
progress_bar = FalconProgressBar(num_iters_per_epoch)

2023-06-23 23:39:06,846	INFO streaming_executor.py:91 -- Executing DAG InputDataBuffer[Input] -> LimitOperator[limit=8000]
2023-06-23 23:39:06,847	INFO streaming_executor.py:92 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-06-23 23:39:06,847	INFO streaming_executor.py:94 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from ray.train.lightning import LightningTrainer, LightningConfigBuilder

# GROUP_SIZE = 1.7e7
GROUP_SIZE = 1e8

deepspeed_configs = {
    "zero_allow_untested_optimizer": True,
    # "fp16": {
    #     "enabled": True,
    #     "initial_scale_power": 8,
    # },
    "bf16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
        # "offload_param": {
        #     "device": "cpu",
        #     "pin_memory": True
        # },
        "overlap_comm": True,
        "contiguous_gradients": True,
        # "sub_group_size": GROUP_SIZE,
        "reduce_bucket_size": HIDDEN_SIZE * HIDDEN_SIZE,
        "stage3_prefetch_bucket_size": 0.9 * HIDDEN_SIZE * HIDDEN_SIZE,
        "stage3_param_persistence_threshold": 10 * HIDDEN_SIZE,
        # "stage3_max_live_parameters": 2e8,
        # "stage3_max_reuse_distance": 2e8,
    },
    # "activation_checkpointing":{
    #     "partition_activations":True,
    #     "cpu_checkpointing":True,
    # },
    # "autotuning": {"enabled": True}
}

lightning_config = (
    LightningConfigBuilder()
    .module(cls=Falcon7BModel)
    .trainer(
        max_epochs=1, accelerator="gpu", precision="bf16-mixed", callbacks=[progress_bar], accumulate_grad_batches=2
    )
    .strategy(
        name="deepspeed",
        # stage=3,
        config=deepspeed_configs
        # offload_optimizer=True,
        # offload_parameters=True,
        # offload_params_device="cpu",
        # offload_optimizer_device="cpu",
        # partition_activations=True,
        # cpu_checkpointing=True,
        # contiguous_gradients=True,
        # reduce_bucket_size=2e8,
        # allgather_bucket_size="auto",
        # sub_group_size="auto",
    )
    .checkpointing(save_top_k=0, save_weights_only=True, save_last=True)
    .build()
)

In [8]:
trainer = LightningTrainer(
    lightning_config=lightning_config,
    run_config=RunConfig(
        name="vicuna-13b-finetune", 
        storage_path="s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/yunxuanx-test/vicuna-test",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            _checkpoint_keep_all_ranks=True,
            _checkpoint_upload_from_workers=True
        ),
    ),
    scaling_config=ScalingConfig(num_workers=NUM_WORKERS, use_gpu=True, resources_per_worker={"CPU": 15, "GPU": 1}),
    datasets={"train": ray_dataset},
    datasets_iter_config={"batch_size": BATCH_SIZE_PER_WORKER},
    preprocessor=preprocessor,
)

2023-06-23 23:39:10,866	WARNING base_trainer.py:201 -- The `preprocessor` arg to Trainer is deprecated. Apply preprocessor transformations ahead of time by calling `preprocessor.transform(ds)`. Support for the preprocessor arg will be dropped in a future release.


In [9]:
trainer.fit()

(pid=5998, ip=10.0.84.83) [2023-06-23 23:39:15,728] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
(download_vicuna_13b pid=42002, ip=10.0.121.227) 15: Finished [repeated 15x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


(LightningTrainer pid=5998, ip=10.0.84.83) The `preprocessor` arg to Trainer is deprecated. Apply preprocessor transformations ahead of time by calling `preprocessor.transform(ds)`. Support for the preprocessor arg will be dropped in a future release.
(LightningTrainer pid=5998, ip=10.0.84.83) Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.
(LightningTrainer pid=5998, ip=10.0.84.83) 
(LightningTrainer pid=5998, ip=10.0.84.83) Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode
(LightningTrainer pid=5998, ip=10.0.84.83) Starting distributed worker processes: ['6053 (10.0.84.83)', '6364 (10.0.95.215)', '6074 (10.0.79.156)', '6110 (10.0.120.86)', '5898 (10.0.66.134)', '6087 (10.0.72.137)', '17232 (

(pid=5998, ip=10.0.84.83) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=5998, ip=10.0.84.83) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(RayTrainWorker pid=6074, ip=10.0.79.156) [2023-06-23 23:39:26,513] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


...

Epoch 0: 100%|██████████| 125/125 [1:16:55<00:00, 36.92s/it, v_num=0, train_loss=1.460]


(RayTrainWorker pid=6110, ip=10.0.120.86) Uploading checkpoint files from worker rank 3 to cloud URI s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/yunxuanx-test/vicuna-test/vicuna-13b-finetune/LightningTrainer_d3e90_00000_0_2023-06-23_23-39-11/checkpoint_000000.
(RayTrainWorker pid=5970, ip=10.0.87.254) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1802: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details. [repeated 15x across cluster]
(RayTrainWorker pid=5970, ip=10.0.87.254)   warnings.warn( [repeated 15x across cluster]
(RayTrainWorker pid=5845, ip=10.0.64.104) Done uploading checkpoint files.
(RayTrainWorker pid=6364, ip=10.0.95.215) Uploading checkpoint files from worker rank 1 to cloud URI s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/yunxuanx-test/vicuna-test/vicuna-13b-finetune/LightningTrainer_d3e90_

Epoch 0: 100%|██████████| 125/125 [1:17:48<00:00, 37.35s/it, v_num=0, train_loss=1.460]


(LightningTrainer pid=5998, ip=10.0.84.83) Uploading trial artifacts took 22.242 s, which may be a performance bottleneck. Consider saving fewer/smaller artifacts to the trial log directory, or disable artifact syncing with `SyncConfig(sync_artifacts=False)`.
2023-06-24 00:59:43,285	INFO tune.py:1148 -- Total run time: 4832.17 seconds (4789.95 seconds for the tuning loop).


Result(
  metrics={'_report_on': 'train_epoch_end', 'train_loss': 1.4609375, 'epoch': 0, 'step': 62, 'should_checkpoint': True, 'done': True, 'trial_id': 'd3e90_00000', 'experiment_tag': '0'},
  path='s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/yunxuanx-test/vicuna-test/vicuna-13b-finetune/LightningTrainer_d3e90_00000_0_2023-06-23_23-39-11',
  checkpoint=LightningCheckpoint(uri=s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/yunxuanx-test/vicuna-test/vicuna-13b-finetune/LightningTrainer_d3e90_00000_0_2023-06-23_23-39-11/checkpoint_000000)
)